# 座標文件檔輸出至影像

說明:  
YOLOv4偵測影片輸出座標文件檔(如附件1)，將座標文件檔結合在原始影片輸出有畫框的影片。

參數:
1. `ROOT`              : 原始影片/座標文件檔路徑
2. `video_save_folder` : 儲存影片路徑
3. `color_file`        : 標籤/顏色清單文件檔 (可有可無)
4. `isAll`             : `isAll=True`:所有標籤畫出來, `isAll=False`:僅將`onlyLabels`內的標籤畫出來
5. `onlyLabels`        : 只要畫特定的標籤 (僅`isAll=False`時定義)

資料夾樹狀圖:
```
ROOT  
├─ video_save_folder  # 程式建立  
│  ├─ 000001_out.avi  # 程式輸出  
│  ├─ 000002_out.avi  # 程式輸出  
│  └─ ..  
├─ 000001.MOV  
├─ 000001.txt  
├─ 000002.MOV  
├─ 000002.txt  
├─ ...  
└─ class_colors.txt  # 標籤顏色清單 (可有可無)  
```

---
備註:
YOLOv4偵測影片輸出座標文件檔指令(in Linux)如:
```
./darknet detector demo ./train.data ./yolov4-tiny.cfg ./best.weights -dont_show -ext_output ./dataset/test/video0001.MOV > ./dataset/result_test/video0001.txt
```

---
附件1:
```
Demo
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
Create CUDA-stream - 0 
 Create cudnn-handle 0 
nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 

 seen 64, trained: 384 K-images (6 Kilo-batches_64) 
video file: ../tmp/test_video/220208150059.MOV
Video stream: 1280 x 720 
Objects:


FPS:0.0 	 AVG_FPS:0.0
Objects:

cone: 98% 	(left_x:  629   top_y:  345   width:   71   height:  104)
cone: 92% 	(left_x:  596   top_y:  264   width:   52   height:  103)
person: 95% 	(left_x:  727   top_y:  175   width:  112   height:  238)

FPS:23.3 	 AVG_FPS:0.0
Objects:

cone: 97% 	(left_x:  629   top_y:  345   width:   70   height:  102)
cone: 92% 	(left_x:  596   top_y:  264   width:   52   height:  102)
person: 96% 	(left_x:  728   top_y:  174   width:  108   height:  240)

FPS:43.5 	 AVG_FPS:0.0
...
```

In [1]:
import os
import cv2
import random

In [2]:
# VARIABLE
ROOT = 'C:\\Users\\danielwu\\Desktop\\txt2video'
video_save_folder = 'videos_out'
color_file = 'class_colors.txt'
isAll = False
if not isAll:
    onlyLabels = ['person']

In [3]:
## @brief Description: 建立資料夾
#  @param folder
#  
#  @return None
#  @date 20220321  danielwu
def MakeDirs(folder):
    if not os.path.isdir(folder):
        os.makedirs(folder)

In [4]:
## @brief Description: 建立標籤對應的顏色
#  @param [in] name          標籤名稱
#  
#  @return 'label:(B, G, R)'
#  @date 20220406  danielwu
def ClassColors(name):
    return '{}:\t{}'.format(name, (
        random.randint(0, 255),
        random.randint(0, 255),
        random.randint(0, 255)))

In [5]:
## @brief Description: 讀取/建立標籤顏色清單
#  @param [in] color_path        標籤顏色清單文件檔
#  @param [in] labels_list       標籤清單
#  @param [in] dict_class_colors {標籤:顏色}
#  
#  @return label_list      標籤清單
#          all_detections  所有偵測訊息
#  @date 20220406  danielwu
def CheckColors(color_path, labels_list, dict_class_colors={}):
    if os.path.isfile(color_path):
        with open(color_path, 'r') as f:
            line = f.readline()
            while line:
                if '#' in line:             # ignore comments
                    line = f.readline()
                    continue
                line = line.split('\n')[0]  # remove '\n' in each line
                if line == '':              # ignore empty line
                    line = f.readline()
                    continue
                dict_class_colors[line.split(':\t')[0]] = eval(line.split(':\t')[1])  # eval() : str to tuple
                line = f.readline()
        for label in labels_list:
            if label not in dict_class_colors:
                with open(color_path, 'a') as f:
                    f.write('\n{}'.format(ClassColors(label)))
                CheckColors(color_path, labels_list, dict_class_colors)
    else:
        with open(color_path, 'w') as f:
            f.write('## ' + '='*21 + '\n')
            f.write('# "label:" + "\\t" + "(B, G, R)"\n')
            f.write('# ' + '='*22 + '\n')
            for label in labels_list:
                f.write('\n{}'.format(ClassColors(label)))
        CheckColors(color_path, labels_list, dict_class_colors)
            
    return dict_class_colors

In [6]:
def GetIndex(lst=None, item=''):
    return [index for (index, value) in enumerate(lst) if value == item]

In [7]:
## @brief Description: 取得母資料夾內所有特定格式檔案路徑
#  @param [in] folder     母資料夾路徑
#  @param [in] extension  副檔名
#  
#  @return paths
#  @date 20220321  danielwu
def GetPaths(folder, extension='.MOV'):
    paths = []

    for response in os.walk(folder):       # response = (dirpath, dirname, filenames)
        if response[2] != []:              # look for filenames
            for f in response[2]:
                if f.endswith(extension):  # only append .MOV file in paths 
                    paths.append(os.path.join(response[0], f))
    
    return paths

In [8]:
## @brief Description: 從YOLOv4物件偵測輸出的文件檔取得偵測的訊息
#  @param [in] txt_path    YOLOv4物件偵測輸出的文件檔
#  
#  @return label_list      標籤清單
#          all_detections  所有偵測訊息
#  @date 20220406  danielwu
def Txt2Detections(txt_path):
    info = []
    with open(txt_path, 'r') as f:
        for line in f.readlines():
            line = line.split('\n')[0]
            if line != '':
                info.append(line)

    flag = False
    str_obj_info = []
    tmp = []
    for message in info:
        if message[:3] == 'Obj':
            flag = True
        if message[:3] == 'FPS':
            flag = False
        if flag:
            if message[:3] != 'Obj':
                tmp.append(message)
        else:
            str_obj_info.append(('Object:', tmp))
            tmp = []

    label_list = []
    all_detections = []
    for idx, obj in enumerate(str_obj_info):
        objs = obj[1]
        if objs != []:
            for obj in objs:
                part = obj.split('\t')
                label = part[0].split(':')[0]
                conf = part[0].split(':')[1].split('%')[0]
                run = part[1].split('left_x:')[1]
                x = run.split('   top_y')[0]
                run = run.split('top_y:')[1]
                y = run.split('   width')[0]
                run = run.split('width:')[1]
                w = run.split('   height')[0]
                run = run.split('height:')[1]
                h = run.split(')')[0]

                bbox = (int(x), int(y), int(w), int(h))
                detections = (label, int(conf), bbox)
                all_detections.append(idx)
                all_detections.append(detections)
                if label not in label_list:
                    label_list.append(label)

    return label_list, all_detections

In [9]:
## @brief Description: YOLOv4 bbox 轉成左上右下點資訊
#  @param [in] bbox    YOLOv4 bbox
#  
#  @return orig_left, orig_top, orig_right, orig_bottom
#  @date 20220406  danielwu
def Convert4Cropping(bbox):
    x, y, w, h = bbox

    orig_left    = int(x)
    orig_right   = int(x + w)
    orig_top     = int(y)
    orig_bottom  = int(y + h)

    return (orig_left, orig_top, orig_right, orig_bottom)

In [10]:
## @brief Description: 透過YOLOv4物件偵測輸出的文件檔將偵測的物件畫在影片上
#  @param [in] video_path     影片路徑
#  @param [in] video_save     儲存影片路徑
#  @param [in] all_detections 所有 bbox 的訊息
#  @param [in] class_colors   各標籤的顏色
#  
#  @return None
#  @date 20220406  danielwu
def Drawing(video_path, video_save, all_detections, class_colors):
    def DrawDetial():
        color = class_colors[label]
        text = '{} {:3d}%'.format(label, conf)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.rectangle(frame, pos1, pos2, color, 2)
        cv2.rectangle(frame, pos3, pos4, color, -1)
        cv2.putText(frame, text, pos5, font, .6, (0, 0, 0), 1, cv2.LINE_AA)
        
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    size = (int(width), int(height))
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    videoWrite = cv2.VideoWriter(video_save, fourcc, fps, size)

    num = 10 # 從第10幀開始畫... (YOLOv4輸出文字檔少10幀)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if num in set(all_detections):
            get_idx = GetIndex(all_detections, num)
            for idx in get_idx:
                label = all_detections[idx+1][0]
                conf = all_detections[idx+1][1]
                bbox = all_detections[idx+1][2]
                left, top, right, bottom = Convert4Cropping(bbox)
                pos1 = (left, top)
                pos2 = (right, bottom)
                pos3 = [left - 1, top + 1]
                pos4 = [left + (len(label)+5)*12, top - 18]
                pos5 = [left + 3, top - 3]
                
                if (left + (len(label)+5)*12) > width:
                    pos3[0] = right - (len(label)+5)*12
                    pos4[0] = right + 1
                    pos5[0] = right + 3 - (len(label)+5)*12
                
                if top < 18:
                    pos3[1] = top -1
                    pos4[1] = top + 18
                    pos5[1] = top + 15
                
                if isAll:
                    DrawDetial()
                else:
                    if label in onlyLabels:
                        DrawDetial()
        videoWrite.write(frame)
        num += 1
    
    cap.release()
    videoWrite.release()

In [11]:
if __name__ == '__main__':
    random.seed(3)
    
    MakeDirs(os.path.join(ROOT, video_save_folder))
    
    txt_paths = GetPaths(ROOT, extension='.txt')
    color_path = os.path.join(ROOT, color_file)
    if os.path.isfile(color_path):
        txt_paths.remove(color_path)
    
    for txt_path in txt_paths:                                                             # ~/123456789_NN_v01.txt
        basename = os.path.basename(txt_path).split('.')[0]                                # 123456789_NN_v01
        video_save = os.path.join(ROOT, video_save_folder, '{}_out.avi'.format(basename))  # ~/123456789_NN_v01_out.MOV
        dirname = os.path.dirname(txt_path)                                                # ~/
        video_basename = basename.split('_')[0]                                            # 123456789
        video_path = os.path.join(dirname, '{}.MOV'.format(video_basename))                # ~/123456789.MOV
        if not os.path.isfile(video_path):
            print('{} does not in {}'.format(os.path.basename(video_path), ROOT))
            continue
        
        labels_list, all_detections = Txt2Detections(txt_path)
        class_colors = CheckColors(color_path, labels_list)
        
        Drawing(video_path, video_save, all_detections, class_colors)

    print('done!')

done!
